In [1]:
#on install les lib de selenium
!pip install selenium
!pip install webdriver_manager


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#tout ce qu'il faut pour bien travailler avec selenium
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome

import tempfile
import time
import datetime

import warnings
warnings.filterwarnings("ignore")

#parametres interractif permet de voir ou pas les actions dans le navigateur
_interractif = False

In [3]:
def verif_page(_driver,_pattern='//body'):
    """ verifie que la page est bien chargée

    Args:
        _driver (webdriver): le webdrive selenium de chargé
        _pattern (string): l element à chercher dans la page
    """
    try:
        elem = WebDriverWait(_driver, 10).until(EC.presence_of_element_located((By.XPATH, _pattern)))
    except Exception as e:
        print('Erreur {}'.format(e))



#on va configurer le navigateur avant son lancement
if _interractif:
    #configuration en mode interractif
    driver = webdriver.Chrome()
else:
    #configuration en mode headless
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-dev-shm-usage')    
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service,options=chrome_options)  #Initialisation du driver 
#on charge la page d'accueil de these
driver.get('https://theses.fr/') #Accès à la page du site
#on va verifier d avoir le retour de la page
verif_page(driver,_pattern='//body')
#on affiche par exemple le titre de la page web pour verifier d etre au bon endroit
print(driver.title)


Theses.fr


In [4]:
#on passe en plein ecran pour avoir toutes les options d'affichage du site
driver.fullscreen_window()
#on va taper dans la barre de recherche la recherche représentation
driver.find_element(By.XPATH,'//input[@id="searchbar"]').send_keys("représentation" )
driver.find_element(By.XPATH,'//i[@id="magnifying-glass"]').click()
#on va verifier d avoir le retour de la page
verif_page(driver,_pattern='//div[@class="result-pagination"]')

#on affiche 50 page, on click sur le chevron
driver.find_element(By.XPATH,'//div[@class="result-pagination"]/div[@class="first-bar-element"]//i[contains(@class,"mdi-chevron-down")]').click()
#on choisi 50
_btn_javascript = driver.find_element(By.XPATH,'//div[@class="v-overlay-container"]//div[@class="v-overlay__content v-select__content"]//div[@class="v-list-item-title" and contains(text(),"50")]/../..')
driver.execute_script('arguments[0].click();',_btn_javascript)

In [5]:
#on recupére les 50 url
#on va verifier d avoir le retour de la page
verif_page(driver,_pattern='//a[@class="first-half"]')

_list_a = driver.find_elements(By.XPATH,'//a[@class="first-half"]')
_tab_url = []
for _elemt in _list_a:
    #on click sur la page web de la these
    _tab_url.append(_elemt.get_attribute("href"))


In [6]:
#pour chaque url on va récupérer les informations que l'on souhaite, ici je ne vais en prendre que 4
for _url in _tab_url[0:3]:
    driver.get(_url)
    #on va verifier d avoir le retour de la page
    verif_page(driver,_pattern='//table[@title="meta-données de la thèse"]')
    _list_info = driver.find_elements(By.XPATH,'//table[@title="meta-données de la thèse"]/tbody/tr')
    print ('############# {} #####################'.format(_tab_url.index(_url)))
    for _detail_info in _list_info:
        print(_detail_info.text)


############# 0 #####################
Auteur / Autrice :  Cyril Labbé
Direction :  Julien Berestycki, Amaury Lambert
Type :  Thèse de doctorat
Discipline(s) :  Mathématiques
Date :  Soutenance en 2013
Etablissement(s) :  Paris 6
############# 1 #####################
Auteur / Autrice :  Marie Aguirre
Direction :  Anne Reboul, Olivier Mascaro
Type :  Thèse de doctorat
Discipline(s) :  Sciences cognitives
Date :  Soutenance le 08/07/2021
Etablissement(s) :  Lyon
Ecole(s) doctorale(s) :  École doctorale Neurosciences et Cognition (NSCo) (Lyon)
Partenaire(s) de recherche :  Laboratoire : Laboratoire Langage Cerveau Cognition (Bron)
établissement opérateur d'inscription : Université Claude Bernard (Lyon ; 1971-....)
Jury :  Président / Présidente : Yves Rossetti
Examinateurs / Examinatrices : Anne Reboul, Olivier Mascaro, Rana Esseily, Nausicaa Pouscoulous, Pierre Jacob
Rapporteurs / Rapporteuses : Rana Esseily, Nausicaa Pouscoulous
############# 2 #####################
Auteur / Autrice :  C

In [7]:
#on ferme proprement le navigateur, la ram c'est important
driver.quit()